# Embedding Agent

What if I trained an agent on word embeddings through reinforcement learning, to predict the results of *semantic equations* à la "king - man = queen"?

Can I study the world model it builds, and thus deduce things about the world models that could be built from the semantic embeddings?
Can I apply IRL methodology to study the reward structure?
Can I use the agents knowledge of semantics to have it act in a (ad hoc and virtual) world?

## Imports

In [11]:
from torchtext.vocab import GloVe as glove

OSError: dlopen(/Users/johndoe/rei/lib/python3.11/site-packages/torchtext/lib/libtorchtext.so, 0x0006): Symbol not found: __ZN2at4_ops15to_dtype_layout4callERKNS_6TensorENSt3__18optionalIN3c1010ScalarTypeEEENS6_INS7_6LayoutEEENS6_INS7_6DeviceEEENS6_IbEEbbNS6_INS7_12MemoryFormatEEE
  Referenced from: <858250ED-4F56-369D-BFB9-938C1A617F97> /Users/johndoe/rei/lib/python3.11/site-packages/torchtext/lib/libtorchtext.so
  Expected in:     <EEA9DBAE-D739-32BF-BF5C-B495A4B0502E> /Users/johndoe/rei/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib

## Embedding space investigations

In [10]:
examples = ['chip', 'baby', 'Beautiful']
vec = glove(name='6B', dim=50)
ret = vec.get_vecs_by_tokens(examples, lower_case_backup=True)


NameError: name 'glove' is not defined